# WORD EMBEDDING

## 参考资料

[什么是 word embedding?](https://www.zhihu.com/question/32275069/answer/61059440)

[WORD EMBEDDINGS: ENCODING LEXICAL SEMANTICS](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#sphx-glr-beginner-nlp-word-embeddings-tutorial-py)

[YJango的Word Embedding--介绍](https://zhuanlan.zhihu.com/p/27830489)

## 概念解析